In [1]:
!pip install sentence-transformers transformers chromadb langchain datasets streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.8 MB/s eta 0:00

In [2]:
!pip install langchain langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [3]:
import sentence_transformers, transformers, chromadb, langchain
from datasets import load_dataset
import re
from chromadb import Client, utils
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import  pipeline

In [4]:
dt = load_dataset("toughdata/quora-question-answer-dataset")
dt

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/485 [00:00<?, ?B/s]

Quora-QuAD.jsonl:   0%|          | 0.00/60.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 56402
    })
})

In [5]:
dt['train'][900]

{'question': 'How can I buy IKEA or IKEA-like furniture in India?',
 'answer': "Ikea is known for its low prices, so it's common to understand if we pay less then can we get the [LINKED_TEXT: best quality] [URL: http://www.thehomedekor.in/], the answer is straight away no. It's not about India they have the same quality level in the whole world, but in developed countries soild wood furniture option is much expensive whereas in India we get soild furniture at best price because its made in India and India is one of the biggest exporters of solid wood furniture in the world. So for us its better to buy soild wood furniture instead of buying ikeas imported Chinese furniture.\n [LINKED_TEXT: To buy modern design like Ikea] [URL: http://www.thehomedekor.in/], The Home Dekor is the best option to buy modern ikea designs in solid wood\n"}

In [6]:
# need to clean data
def clean_text(text: str) -> str:
    # a) remove [LINKED_TEXT: ...] and [URL: ...]
    text = re.sub(r'\[LINKED_TEXT:.*?\]', '', text)
    text = re.sub(r'\[URL:.*?\]', '', text)
    # b) collapse (spaces, newlines, tabs) into one space
    text = re.sub(r'\s+', ' ', text)
    # c) strip leading/trailing spaces
    return text.strip()

def preprocess(example):
    example['question'] = clean_text(example['question'])
    example['answer']   = clean_text(example['answer'])
    return example

cleaned_dt = dt['train'].map(preprocess)
print(cleaned_dt[0])


Map:   0%|          | 0/56402 [00:00<?, ? examples/s]

{'question': 'Why whenever I get in the shower my girlfriend want to join?', 'answer': 'Isn’t it awful? You would swear that there wasn’t enough hot water to go around!'}


In [7]:
cleaned_dt[900]

{'question': 'How can I buy IKEA or IKEA-like furniture in India?',
 'answer': "Ikea is known for its low prices, so it's common to understand if we pay less then can we get the , the answer is straight away no. It's not about India they have the same quality level in the whole world, but in developed countries soild wood furniture option is much expensive whereas in India we get soild furniture at best price because its made in India and India is one of the biggest exporters of solid wood furniture in the world. So for us its better to buy soild wood furniture instead of buying ikeas imported Chinese furniture. , The Home Dekor is the best option to buy modern ikea designs in solid wood"}

In [8]:
QA_txt=[]
for example in cleaned_dt:
    qa_string=example['question'] + " " + example['answer']
    QA_txt.append(qa_string)

QA_txt[0]


'Why whenever I get in the shower my girlfriend want to join? Isn’t it awful? You would swear that there wasn’t enough hot water to go around!'

In [9]:
model = sentence_transformers.SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
single_vector = model.encode(QA_txt[0])
single_vector.shape

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


(384,)

In [10]:
txt_emb = model.encode(QA_txt, batch_size=32, normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/1763 [00:00<?, ?it/s]

In [15]:
in_memory=True
client = Client()
default_collection_settings= {

    "embedding_function": embedding_functions.SentenceTransformerEmbeddingFunction(model_name='sentence-transformers/all-MiniLM-L6-v2')

}

In [16]:
ids = [f"q{i}" for i in range(len(QA_txt))]
metas = [{"question": example["question"], "answer": example["answer"]} for example in cleaned_dt]
batch_size = 1000


In [22]:
documents = [f"Q: {example['question']} A: {example['answer']}" for example in cleaned_dt]
collection = client.get_or_create_collection(name="quora_qa")

for i in range(0, len(ids), batch_size):
    batch_ids = ids[i:i+batch_size]
    batch_metas = metas[i:i+batch_size]
    batch_docs = documents[i:i+batch_size]
    batch_embs = txt_emb[i:i+batch_size]
    collection.upsert(
        ids=batch_ids,
        documents=batch_docs,
        embeddings=batch_embs,
        metadatas=batch_metas
    )


In [23]:
existing = client.list_collections()
print(existing)

[Collection(name=quora_qa)]


In [24]:
User_query = "best persian restaurant?"
UQ_emb = model.encode(User_query)
results = collection.query(
    query_embeddings=UQ_emb,
    n_results=5
)

In [25]:
print(results["metadatas"][0][0]["question"])
print(results["metadatas"][0][0]["answer"])

What is absolutely the best Indian restaurant in the world, and why?
If you are seeking for a best Indian Restaurant in Bahrain. Your search is over with "Al Wamiya Restaurant” Is the best Indian restaurant in all over the world Al Wasmiya Provide best Indian food Items and also they Provide Arabic dishes. Al Wasmiya World Famous for his tastiest Dam Pukth Biryani. For more information You can Visit Our Website


In [26]:
# time to evaluate:
dt_validation= cleaned_dt.shuffle(seed=42).select(range(1000))
dt_validation

Dataset({
    features: ['question', 'answer'],
    num_rows: 1000
})

In [27]:
test_queries = [example["question"] for example in dt_validation]
test_answers = [example["answer"] for example in dt_validation]


In [28]:
K = 5

correct_at_1 = 0
correct_at_5 = 0

for i, query in enumerate(test_queries):
    UQ_emb = model.encode(query)

    results = collection.query(
        query_embeddings=UQ_emb,
        n_results=K
    )

    retrieved_answers = [results["metadatas"][0][j]["answer"] for j in range(K)]

    ground_truth = test_answers[i]

    if retrieved_answers[0] == ground_truth:
        correct_at_1 += 1

    if ground_truth in retrieved_answers:
        correct_at_5 += 1

total_queries = len(test_queries)
precision_at_1 = correct_at_1 / total_queries
precision_at_5 = correct_at_5 / total_queries

print(f"Precision@1: {precision_at_1:.3f}")
print(f"Precision@5: {precision_at_5:.3f}")



Precision@1: 0.053
Precision@5: 0.238


In [29]:
# Not a great result but enough for the small model
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

collection_name = "quora_qa"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

persist_directory = None

vectorstore = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    persist_directory=persist_directory
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


/tmp/ipython-input-1825630329.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
/tmp/ipython-input-1825630329.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [30]:
llm_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-425032360.py:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [31]:
# the Retrieval QA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)


In [32]:
# Example user query
user_question = "What is the best way to learn Python as a beginner?"
result = rag_chain(user_question)
print("Generated answer:", result['result'])

for i, doc in enumerate(result['source_documents']):
    print(f"\n--- Source #{i+1} ---")
    print("Q:", doc.metadata['question'])
    print("A:", doc.metadata['answer'])


/tmp/ipython-input-1383358897.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain(user_question)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Token indices sequence length is longer than the specified maximum sequence length for this model (740 > 512). Running this sequence through the model will result in indexing errors


Generated answer: Make fun projects in the various languages you are learning.

--- Source #1 ---
Q: I have a fairly basic understanding of Python. But now I want to learn C++. Does this require me to learn C first?
A: Yes, because C++ is essentially the ++ addon to C. But you do not need to learn C to beautiful mastery. Stuff like union, pointer arithmetic, bitwise variables are of no use to modern computing anyway. C level overview understanding for syntax and skeletal awareness of how and what comprise a program is sufficient to go on to C++.

--- Source #2 ---
Q: I have a fairly basic understanding of Python. But now I want to learn C++. Does this require me to learn C first?
A: You do not have to C first to learn C++. I would say your basic understanding of Python is better at understand to use C++ at a higher level then learning C.

--- Source #3 ---
Q: I have a fairly basic understanding of Python. But now I want to learn C++. Does this require me to learn C first?
A: Not sure t